In [1]:
import sys
import os
from os.path import abspath
import numpy as np
import pandas as pd
from utils.generate_network import generate_network
from utils.prepare_data import prepare_data
from utils.popphy_io import save_params, load_params
from utils.popphy_io import get_stat, get_stat_dict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from models.PopPhy import PopPhyCNN
import warnings
from datetime import datetime
import json
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
#from models.PopPhy2 import ResNet
from models.PopPhy2 import ResNet

# Data Preparation

### Reading Configuration
Configuring which data to read in, minimun threshold needed in an OTU (individual sample must have at least set threshold relative abundance), and how many k folds for k fold cross validation.

In [2]:
dataset = 'CDr'
threshold = 0
k = 5

### Reduce Features
Reduce amount of OTU features by filtering out OTUs that contain no individual sample with a relative abundance greater than the set threshold.

In [3]:
path = "../data/" + dataset
data = pd.read_csv(path + '/abundance.tsv', index_col=0, sep='\t', header=None)
to_drop = data.loc[(data < threshold).all(axis=1)]
data = data.drop(to_drop.index)

data

,1,2,3,4,5,6,7,8,9,10,...,106,107,108,109,110,111,112,113,114,115
0,,,,,,,,,,,,,,,,,,,,,
k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter,0.000000,0.000000,0.000035,0.000084,0.000943,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanosphaera,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
k__Bacteria.....,0.000000,0.000000,0.000313,0.000000,0.000021,0.000034,0.0,0.000072,0.000000,0.000000,...,0.000000,0.000000,0.000035,0.000000,0.000000,0.000000,0.000127,0.000000,0.000000,0.000000
k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Actinomycetales|f__Actinomycetaceae|g__,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Actinomycetales|f__Actinomycetaceae|g__Actinomyces,0.000068,0.000000,0.000000,0.000042,0.000172,0.000000,0.0,0.000024,0.000000,0.000000,...,0.000021,0.000239,0.000035,0.000000,0.000101,0.000269,0.000042,0.000246,0.000000,0.000164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
k__Bacteria|p__Tenericutes|c__Mollicutes|o__Anaeroplasmatales|f__Anaeroplasmataceae|g__,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
k__Bacteria|p__Tenericutes|c__Mollicutes|o__RF39|f__|g__,0.004988,0.000921,0.014342,0.002366,0.015264,0.009125,0.0,0.000361,0.000989,0.000334,...,0.000211,0.000040,0.000565,0.000849,0.000371,0.000067,0.000423,0.000443,0.000162,0.000023
k__Bacteria|p__Tenericutes|c__RF3|o__ML615J.28|f__|g__,0.000046,0.000000,0.000000,0.000000,0.000064,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Create 2d Matrix Representing OTU Data
Dai et al. PopPhy-CNN's (2019) algorithm creates Phylogenetic tree from OTUs and populates tree based on OTU abundances. This tree graph structure is then converted to a 2d Matrix by taking each parent node in the tree graph and pushing them all to the left and childrens' nodes in the same order from left to right the parents were ordered.

In [4]:
my_maps, raw_x, tree_x, raw_features, tree_features, labels, label_set, g, feature_df = prepare_data(path, data)

# norms = np.linalg.norm(my_maps, axis=2, keepdims=True)
# my_maps = my_maps / norms
pd.DataFrame(my_maps[0])

(257, 115)
There are 257 raw features...
Building tree structure...
Found tree file...
Populating trees...
There are 359 tree features...


,0,1,2,3,4,5,6,7,8,9,...,100,101,102,103,104,105,106,107,108,109
0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.002694,0.007485,0.000000,0.120929,0.000000,0.868891,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.002694,0.000392,0.001497,0.000184,0.005412,0.000000,0.120929,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.002694,0.000392,0.000000,0.001474,0.000023,0.000046,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
5,0.0,0.002694,0.000000,0.000000,0.000392,0.000000,0.000000,0.000000,0.000046,0.001428,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
6,0.0,0.002694,0.000000,0.000000,0.000000,0.000392,0.000000,0.000000,0.000046,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
7,0.0,0.000000,0.000000,0.000392,0.000000,0.000000,0.000000,0.000000,0.000000,0.005297,...,0.0,0.0,0.000092,0.0,0.0,0.000069,0.0,0.0,0.0,0.0
8,0.0,0.005297,0.088204,0.000000,0.000000,0.000092,0.000138,0.000000,0.000760,0.001175,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


### Creating training and test sets
Splitting data into k training and k test sets

In [5]:
#one hot encoding
input = my_maps
target = tf.keras.utils.to_categorical(labels, 2, dtype='int64')
    

#shuffle dataset
seed = np.random.randint(100)
# np.random.seed(seed)
# np.random.shuffle(input)
np.random.seed(seed)
np.random.shuffle(target)

np.random.seed(seed)
np.random.shuffle(my_maps)
np.random.seed(seed)
np.random.shuffle(raw_x)
np.random.seed(seed)
np.random.shuffle(tree_x)
np.random.seed(seed)
np.random.shuffle(labels)


#create k training and k test sets
groups_input = []
groups_target = []
k_size = len(input)//k
start, end = 0, k_size
for i in range(k):
    if i == k-1:
        group_input = input[start:]
        group_target = target[start:]
    else:
        group_input = input[start:end]
        group_target = target[start:end]
    start += k_size
    end += k_size
    groups_input.append(group_input)
    groups_target.append(group_target)

x_train = []
y_train = []
x_test = []
y_test = []
for i in range(k-1, -1, -1):
    x_train.append(np.concatenate((groups_input[i-1], groups_input[i-2], groups_input[i-3], groups_input[i-4])))
    y_train.append(np.concatenate((groups_target[i-1], groups_target[i-2], groups_target[i-3], groups_target[i-4])))

    x_test.append(groups_input[i])
    y_test.append(groups_target[i])

# Model

### Training model
Data is log transformed and then a MinMax transformation. Uses CNN that employs skipped residual identity blocks borrowed from the classic ResNet model then a FC Neural Network to make phenotype prediction. Model dimensions printed below.

In [6]:
data_lst = []

# for i in range(k):
#     x_train1 = x_train[i]
#     y_train1 = y_train[i]
#     x_test1 = x_test[i]
#     y_test1 = y_test[i]

#     model = ResNet(height = x_train1.shape[1], width = x_train1.shape[2], channels = 1, classes = 2)
#     model.init_model()

#     model.train(x_train1, y_train1, x_test1, y_test1, dataset, use_weights = False)
#     y_pred = model.predict(x_test1)
#     auc_roc, auc_pr, f1, mcc = model.evaluate(y_test1, y_pred)
#     data_lst.append([auc_roc, auc_pr, f1, mcc])
    
#     #model.model.save_weights(path + "/model_weights.h5")

#     print(y_test1)
#     print(y_pred)
    
# print(model.model.summary())



n_values = np.max(labels) + 1
labels_oh = np.eye(n_values)[labels]
tree_row = my_maps.shape[1]
tree_col = my_maps.shape[2]

skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
fold = 0
for train_index, test_index in skf.split(my_maps, labels):
    train_x, test_x = my_maps[train_index,:,:], my_maps[test_index,:,:]
    train_y, test_y = labels_oh[train_index,:], labels_oh[test_index,:]
        
    train_x = np.log(train_x + 1)
    test_x = np.log(test_x + 1)
        
    c_prob = [0] * len(np.unique(labels))
    train_weights = []

    for l in np.unique(labels):
        a = float(len(labels))
        b = 2.0 * float((np.sum(labels==l)))
        c_prob[int(l)] = a/b

    c_prob = np.array(c_prob).reshape(-1)

    for l in np.argmax(train_y, 1):
        train_weights.append(c_prob[int(l)])
    train_weights = np.array(train_weights)
        
    scaler = MinMaxScaler().fit(train_x.reshape(-1, tree_row * tree_col))
    train_x = np.clip(scaler.transform(train_x.reshape(-1, tree_row * tree_col)), 0, 1).reshape(-1, tree_row, tree_col)
    test_x = np.clip(scaler.transform(test_x.reshape(-1, tree_row * tree_col)), 0, 1).reshape(-1, tree_row, tree_col)

    train = [train_x, train_y]
    test = [test_x, test_y]

    x_train1 = train_x
    y_train1 = train_y
    x_test1 = test_x
    y_test1 = test_y
        
#         y_train1 = train_y
#         y_test1 = test_y
        
#         x_train1 = np.zeros(train_x.shape)
#         x_train1[train_x != 0] = 1
        
#         x_test1 = np.zeros(test_x.shape)
#         x_test1[test_x != 0] = 1
        
        # for i in range(len(train_x)):
        #     for j in range(len(test_x)):
        #         if np.array_equal(train_x[i], test_x[j]):
        #             print('train')
        #             print(train_x[i])
        #             print('test')
        #             print(test_x[j])
        
        
    model = ResNet(height = train_x.shape[1], width = train_x.shape[2], channels = 1, classes = 2)
    model.init_model()
    model.train(train_x, train_y, test_x, y_test1, dataset, use_weights = False)
    y_pred = model.predict(test_x)
    auc_roc, auc_pr, f1, mcc = model.evaluate(test_y, y_pred)
    data_lst.append([auc_roc, auc_pr, f1, mcc])
    #model.model.save_weights(path + "/model_weights.h5")
    print(test_y)
    print(y_pred)
    print(model.model.summary())
    
    fold += 1
#run += 1
        
        
        
        

Metal device set to: Apple M1 Pro
Epoch 1/150
16/16 [==============================] - 3s 93ms/step - loss: 2.6241 - accuracy: 0.6522 - val_loss: 2.2536 - val_accuracy: 0.6522
Epoch 2/150
16/16 [==============================] - 0s 31ms/step - loss: 1.9390 - accuracy: 0.6739 - val_loss: 1.7595 - val_accuracy: 0.6522
Epoch 3/150
16/16 [==============================] - 0s 30ms/step - loss: 1.4873 - accuracy: 0.7391 - val_loss: 1.4568 - val_accuracy: 0.6522
Epoch 4/150
16/16 [==============================] - 0s 30ms/step - loss: 1.1878 - accuracy: 0.8043 - val_loss: 1.2778 - val_accuracy: 0.6522
Epoch 5/150
16/16 [==============================] - 1s 34ms/step - loss: 0.9657 - accuracy: 0.8913 - val_loss: 1.1599 - val_accuracy: 0.6522
Epoch 6/150
16/16 [==============================] - 0s 31ms/step - loss: 0.8014 - accuracy: 0.9348 - val_loss: 1.0893 - val_accuracy: 0.6522
Epoch 7/150
16/16 [==============================] - 0s 29ms/step - loss: 0.6779 - accuracy: 0.9783 - val_loss: 1.

# Displaying Accuracy Metrics and Saving Metrics

Option to save results of all k folds and weights of last model into same directy as data.

In [7]:
col = [str(i) for i in range(1,k+1)]    
results_df = pd.DataFrame(data_lst, columns = ['auc(roc)', 'auc(pr)', 'f1', 'mcc'])
results_df = results_df.transpose()
results_df.columns = col

#results_df.to_csv(path + "/results.csv")
results_df

,1,2,3,4,5
auc(roc),0.733333,0.933333,0.791667,0.821429,0.723214
auc(pr),0.755050,0.929231,0.776096,0.846800,0.744367
f1,0.690416,0.819099,0.744306,0.570792,0.777541
mcc,0.310530,0.605598,0.464372,0.000000,0.467801


# Saving Model Weights

Option to save model weights of last model in k fold into same directy as data.

In [8]:
#model.model.save_weights(path + "/model_weights.h5")

## 